In [5]:
import pandas as pd
df = pd.read_csv('region_annotations.tsv',sep = '\t')

In [6]:
df
required_df = df.iloc[:, :2]  ##removing all columns except identity_term and token
required_df

,identity_term,token
0,arunachali,athlete
1,arunachali,boxer
2,arunachali,business
3,arunachali,calm
4,arunachali,ceo
...,...,...
2551,uttarakhandi,travel
2552,uttarakhandi,treasurer
2553,uttarakhandi,umpire
2554,uttarakhandi,weak


In [7]:
token_counts = required_df.groupby('token')['identity_term'].nunique()

# Finding tokens that are common across all categories
common_tokens = token_counts[token_counts == required_df['identity_term'].nunique()].index

# Constructing a new dataset containing rows with common tokens
new_df = required_df[required_df['token'].isin(common_tokens)]

# Printing the new dataset
print(new_df)

        identity_term      token
59         arunachali       poor
64         arunachali  secretary
119          assamese       poor
201          assamese  secretary
269           bengali  secretary
331           bengali       poor
404            bihari  secretary
456            bihari       poor
541      chattisgarhi       poor
548      chattisgarhi  secretary
634              goan       poor
644              goan  secretary
771          gujarati       poor
781          gujarati  secretary
856          haryanvi       poor
861          haryanvi  secretary
890         himachali       poor
891         himachali  secretary
958        jharkhandi       poor
1004       jharkhandi  secretary
1072        kannadiga  secretary
1094        kannadiga       poor
1200         kashmiri       poor
1212         kashmiri  secretary
1313  madhya pradeshi  secretary
1339  madhya pradeshi       poor
1415         manipuri       poor
1430         manipuri  secretary
1552          marathi       poor
1561      

In [9]:
final_df = required_df.index.difference(new_df.index)
print(required_df.loc[final_df])

     identity_term      token
0       arunachali    athlete
1       arunachali      boxer
2       arunachali   business
3       arunachali       calm
4       arunachali        ceo
...            ...        ...
2551  uttarakhandi     travel
2552  uttarakhandi  treasurer
2553  uttarakhandi     umpire
2554  uttarakhandi       weak
2555  uttarakhandi     writer

[2506 rows x 2 columns]


In [12]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained BERT model for masked language modeling
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

def predict_masked_tokens(prompt, top_k=5):
    first_word = final_df['identity_term'].iloc[1] 
    prompt = prompt.replace('[MASK]', first_word)  

    tokenized_text = tokenizer.tokenize(prompt)
    mask_index = tokenized_text.index(first_word)  
    input_ids = tokenizer.convert_tokens_to_ids(tokenized_text)
    input_tensor = torch.tensor(input_ids).unsqueeze(0) 
    with torch.no_grad():
        outputs = model(input_tensor)
    masked_token_logits = outputs.logits[0, mask_index]
    top_k_probabilities, top_k_indices = torch.topk(masked_token_logits, top_k, dim=0)
    top_k_tokens = tokenizer.convert_ids_to_tokens(top_k_indices.tolist())
    probabilities = torch.softmax(top_k_probabilities, dim=0).tolist()

    for token, prob in zip(top_k_tokens, probabilities):
        print(f"{token}: {prob*100:.2f}%")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:

prompt = "bihari are [MASK]."
predict_masked_tokens(prompt,5)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices